# Solutions to sheet 1

Import the necessary libraries: `numpy`, `matplotlib` and `scipy`. We can use the `as` keyword to define shortcuts to access the libraries (example: `import numpy as np` means we can access numpy functions with the `np.` syntax).

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as spo

Set some plotting options – don't worry, this is just to make the plots look prettier.

In [ ]:
## - - - - - - - - - - - - - - - - - - - - - - - - - - -
##      plotting properties
## - - - - - - - - - - - - - - - - - - - - - - - - - - -

mpl.rcParams.update({
    "axes.autolimit_mode":"round_numbers",
    "axes.axisbelow":"False",
    "axes.edgecolor":"000000",
    "axes.facecolor":"FFFFFF",
    "axes.formatter.limits":"-2, 4",
    "axes.formatter.use_mathtext":"True",
    "axes.grid":"False",
    "axes.labelcolor":"000000",
    "axes.labelsize":"12",
    "axes.linewidth":"1",
    "axes.titlesize":"12",
    "axes.xmargin":"0.",
    "axes.ymargin":"0.",
    "errorbar.capsize":"1",
    "figure.autolayout":"True",
    "figure.dpi":"96",
    "figure.edgecolor":"0.50",
    "figure.facecolor":"FFFFFF",
    "figure.figsize":"6, 4.5",
    "font.family":"monospace",
    "font.size":"12",
    "legend.fancybox":"True",
    "legend.framealpha":"0.25",
    "legend.frameon":"True",
    "lines.markersize":"6",
    "savefig.dpi":"150",
    "savefig.facecolor":"FFFFFF00",
    "xtick.bottom":"True",
    "xtick.color":"000000",
    "xtick.direction":"in",
    "xtick.major.size":"10",
    "xtick.minor.size":"5",
    "xtick.top":"True",
    "ytick.color":"000000",
    "ytick.direction":"in",
    "ytick.left":"True",
    "ytick.major.size":"10",
    "ytick.minor.size":"5",
    "ytick.right":"True",
})

## Drawing random numbers from a PDF

First step: Initialise a random number generator.

In [ ]:
gen = np.random.default_rng()

Drawing numbers from a uniform distribution in the interval $[0, 1)$:

In [ ]:
values = gen.uniform(0, 1, 100)

In [ ]:
values[:10]

Next step: plotting the drawn values.

In [ ]:
plt.hist(values, bins=20, range=(0, 1))

plt.show()

Let's do the same with a Gaussian distribution, centred around 15.

In [ ]:
values = gen.normal(15, 1, 1000)

In [ ]:
values[:10]

Now let's plot them again.

In [ ]:
plt.hist(values, bins=50, range=(10, 20))

plt.show()

## Defining and plotting analytical functions

Now let's plot the analytical function as well.

In [ ]:
def gauss(x, mu, sigma):
    pref = 1 / sigma / np.sqrt(2 * np.pi)
    return pref * np.exp(- pow(x - mu, 2) / 2 / sigma / sigma)

In [ ]:
xfunc = np.linspace(10, 20, 101)

In [ ]:
yfunc = gauss(xfunc, 15, 1)

Now let's plot the function and the drawn values into the same figure:

In [ ]:
plt.hist(values, bins=50, range=(10, 20), density=True, label="drawn values")
plt.plot(xfunc, yfunc, label="analytical function", lw=4)
plt.legend()

plt.show()

## Fitting functions to data

Let's use the previously generated data and fit a Gaussian function to it, to get estimators for mu and sigma. First we need to get the bin edges and contents.

In [ ]:
bin_entries, bin_edges, _ = plt.hist(values, bins=50, range=(10, 20), density=True)

plt.show()

In [ ]:
bin_entries

In [ ]:
bin_edges

Calculate the bin centres.

In [ ]:
bin_centres = np.array([0.5 * (bin_edges[i] + bin_edges[i+1]) for i in range(len(bin_edges)-1)])

In [ ]:
bin_centres

Now perform the actual fit. Save the optimized parameter values into `popt`.

In [ ]:
popt, pcov = spo.curve_fit(gauss, bin_centres, bin_entries)

In [ ]:
print(popt)

Ok, this doesn't seem to work ... Let's add some bounds to the parameters: $\mu \in [12, 17]$ and $\sigma \in [0.1, 5.0]$.

In [ ]:
popt, pcov = spo.curve_fit(gauss, bin_centres, bin_entries, bounds=([12, 0.1], [17, 5]))

In [ ]:
print(popt)

This worked!

For comparison, plot the histogram and the fitted curve again.

In [ ]:
plt.hist(values, bins=50, range=(10, 20), density=True, label="drawn values")
plt.plot(bin_centres, gauss(bin_centres, *popt), lw=4,
         label='fit: mu=%.2f, sigma=%.2f' % tuple(popt))
plt.legend()

plt.show()

# Bonus question

In [ ]:
def monty_hall(switch_door=False, print_debug=False):
    """Simulates the Monty Hall problem. Returns Win/Lose boolean.
    
    That is: choses a random door for the player, then choses another random 
    door to reveal (which is neither the winning door nor the door chosen by
    the player). Then, with the 'switch_door' switch, let's the player switch
    from the intially chosen door to the other door that is still closed.
    Returns a Win/Lose boolean.

    """
    doors = ["A", "B", "C"]
    winning_door = np.random.choice(doors)
    chosen_door = np.random.choice(doors)

    # Doors the host can choose from for the reveal.
    reveal_choices = doors.copy()
    reveal_choices.remove(winning_door)
    if chosen_door != winning_door:
        reveal_choices.remove(chosen_door)

    revealed_door = np.random.choice(reveal_choices)
    
    if print_debug:
        print(f"Chosen door:   {chosen_door}")
        print(f"Winning door:  {winning_door}")
        print(f"Revealed door: {revealed_door}")
    
    if switch_door:
        chosen_door = list(set(doors).difference(set(revealed_door)).difference(set(chosen_door)))[0]
        if print_debug:
            print(f"New choice:    {chosen_door}")

    return chosen_door == winning_door

In [ ]:
monty_hall(switch_door=True, print_debug=True)

In [ ]:
monty_hall(switch_door=False, print_debug=True)

In [ ]:
n_experiments = 20_000

experiments = [monty_hall(switch_door=True) for i in range(n_experiments)]
print(f"When switching doors after reveal: {experiments.count(True) / n_experiments}")

experiments = [monty_hall(switch_door=False) for i in range(n_experiments)]
print(f"When *not* switching doors:        {experiments.count(True) / n_experiments}")